# Challenge from [KQL Detective](https://detective.kusto.io/) - "The rarest book is missing!"

I solved this challenge using KQL and received a badge on [Credly](https://www.credly.com/users/dina-levashova).
I wanted to see if I could recreate the solution in Python and evaluate which approach is simpler.

The data for this project can be downloaded here: 

Shelves data: https://kustodetectiveagency.blob.core.windows.net/digitown-books/shelves.csv.gz

Books data: https://kustodetectiveagency.blob.core.windows.net/digitown-books/books.csv.gz

### Prompt

This was supposed to be a great day for Digitown’s National Library Museum and all of Digitown.
The museum has just finished scanning more than 325,000 rare books, so that history lovers around the world can experience the ancient culture and knowledge of the Digitown Explorers.
The great book exhibition was about to re-open, when the museum director noticed that he can't locate the rarest book in the world:
"De Revolutionibus Magnis Data", published 1613, by Gustav Kustov.
The mayor of the Digitown herself, Mrs. Gaia Budskott - has called on our agency to help find the missing artifact.

Luckily, everything is digital in the Digitown library:
- Each book has its parameters recorded: number of pages, weight.
- Each book has RFID sticker attached (RFID: radio-transmitter with ID).
- Each shelve in the Museum sends data: what RFIDs appear on the shelve and also measures actual total weight of books on the shelve.

Unfortunately, the RFID of the "De Revolutionibus Magnis Data" was found on the museum floor - detached and lonely.
Perhaps, you will be able to locate the book on one of the museum shelves and save the day?

### Setup queries provided:
```
.execute database script <|
// Create table for the books
.create-merge table Books(rf_id:string, book_title:string, publish_date:long, author:string, language:string, number_of_pages:long, weight_gram:long)
// Import data for books
// (Used data is utilzing catalogue from https://github.com/internetarchive/openlibrary )
.ingest into table Books ('https://kustodetectiveagency.blob.core.windows.net/digitown-books/books.csv.gz') with (ignoreFirstRecord=true)
// Create table for the shelves
.create-merge table Shelves (shelf:long, rf_ids:dynamic, total_weight:long) 
// Import data for shelves
.ingest into table Shelves ('https://kustodetectiveagency.blob.core.windows.net/digitown-books/shelves.csv.gz') with (ignoreFirstRecord=true)
```

### Question: Which shelf is the book on?

## My solution in KQL:

I examined the contents of each table created:
```
Books 
| take 20 

Shelves
| take 20
```
The "Books" table contained fields including the book's title (book_title), RFID (rf_id) and weight (weight_gram). The "Shelves" table had fields for the unique shelf number (shelf), RFIDs on the shelf (rf_ids) and total weight on shelf (total_weight).

I searched for the missing book by name in the Books table:

    Books | where book_title == "De Revolutionibus Magnis Data"

And found that the book has an RFID of 0cbc13e0aa7d487e8e797d3de3823161 and a weight of 1,764 grams.

I searched the Shelves table for this RFID and found that it is indeed missing:

    Shelves 
    | where rf_ids has "0cbc13e0aa7d487e8e797d3de3823161"

The only way to find the book would be by weight as that is the only other common parameter between Books and Shelves. 

I needed to find the shelf whose total weight was greater than the weight of the books on it.

Testing the code on a sample of 20 to make sure that it works first, I expanded the array of rf_ids that are listed in Shelves to have one RFID per row and called the column "rf_id". I then joined the table with fields in Books based on the corresponding rf_id.
    
    Shelves
    | take 20
    | mv-expand rf_id = rf_ids
    | join kind = leftouter Books on rf_id
    
At this point, I received an error on the "join" operator that the "rf_id" columns are of different types. So I edited the query to convert the type to string in mv-expand:

    Shelves
    | take 20
    | mv-expand rf_id = rf_ids
    | join kind = leftouter Books on rf_id
    
Next, I calculated the weight of all books on each shelf:

    | summarize sum (weight_gram) by shelf, total_weight

This seemed to work, so I ran the query on all values and searched for an instance where the sum of the book weights on each shelf did not equal the total weight:

    Shelves
    // | take 20
    | mv-expand rf_id = rf_ids to typeof(string)
    | join kind = leftouter Books on rf_id
    | summarize sum (weight_gram) by shelf, total_weight
    | where total_weight !=  sum_weight_gram

Unfortunately, I received many results back as there were small discrepancies on most shelves. The numbers seemed to differ by less than 100 grams and since the missing book weighed over 1000 grams, I searched with this criteria:

    Shelves
    // | take 20
    | mv-expand rf_id = rf_ids to typeof(string)
    | join kind = leftouter Books on rf_id
    | summarize sum (weight_gram) by shelf, total_weight
    | where total_weight - sum_weight_gram >= 1000    

The only result was shelf 4242, which had a discrepancy of 1,798 between the total weight and weight of the books listed on it, matching closely to the missing book weight of 1,764.

## My solution in Python:

I decided to repeat the challenge using Python.

I first imported the relevant modules:

In [1]:
import pandas as pd
import gzip # for unzipping csv file
import ast # for converting list-formatted string to list type

missing_title = "De Revolutionibus Magnis Data"

I then unzipped each file and read them as Pandas DataFrames into the variables "books" and "shelves":

In [2]:
def unzip(file):
    with gzip.open(file) as csv:
        return pd.read_csv(csv)

books = unzip('books.csv.gz')
shelves = unzip('shelves.csv.gz')

Looking at first five rows, I checked that import worked:

In [3]:
books.head()

,rf_id,title,publish_date,author,language,number_of_pages,weight_gram
0,b9b84d99512b4d3183d0640e665399a9,Llinell i'r byd,1791,NaN,wel,12,97
1,8a9f7d8f2eb843de8f8f5ea067caf2f1,"Sterbensspiegel, das ist, Sonnenklare Vorstell...",1650,vor disem angefangen durch Rüdolffen Meyern .....,ger,166,1037
2,e92739d76b214402bf2292a2acbe6a22,"Le primier volume del promptuarie, ou repertor...",1614,NaN,roa,286,1814
3,f3a85b14a6db4d458a820149eff13760,Praelectiones de duobus in theologia controver...,1631,per Reverendissimum virum Joan. Davenantium ...,lat,641,3884
4,6b5c01f5aa50469b9fec848851490928,A treatise on the laws concerning the election...,1773,NaN,eng,397,2382


In [4]:
shelves.head()

,shelf_id,book_rf_ids,total_weight_gram
0,1395,"[""e0e068b3ada347cb8ed99b5172fdedf4"",""d748c9ade...",54639
1,4251,"[""c1ef0f12eeab45ef9332c6f6ad8a097a"",""25e1a5179...",51936
2,3589,"[""1b193ac7dfc64213b2f6387f68b0f017"",""a45b03bc4...",83912
3,1833,"[""936e27c5802946e3b1d4ade126e4eb0b"",""9192b2d51...",55231
4,10498,"[""b2cd51f2d0c247b5a9e9950544d077e9""]",209270


I found the missing book information in the "books" DataFrame:

In [5]:
books[books["title"] == missing_title]

,rf_id,title,publish_date,author,language,number_of_pages,weight_gram
280418,0cbc13e0aa7d487e8e797d3de3823161,De Revolutionibus Magnis Data,1613,Gustav Kustov,lat,256,1764


I saved the RFID of missing book in "missing_rfid" variable:

In [6]:
missing_rfid_list = books["rf_id"].where(books["title"] == missing_title)
missing_rfid = missing_rfid_list.dropna().item()

The following function would be used when the correct shelf was identified:

In [7]:
def shelf_found(shelf):
    print("Book found on shelf %d!" % shelf)

I checked that book wasn't in the "shelves" database, based on the book's RFID:

In [8]:
for rfids in shelves["book_rf_ids"]:
    if missing_rfid in rfids:
        shelf = shelves["shelf_id"].where(shelves["book_rf_ids"] == rfids).dropna().item()
        shelf_found(shelf)
        break

I tried to loop through each of the rfids, sum the corresponding book weights, and filter the one where the difference is over 1000:

```
for rfids in shelves.book_rf_ids:
    
    weights = []
    
    for rfid in rfids:
        weight = books["weight_gram"].where(books["rf_id"] == rfid).dropna().item()
        weights.append(weight)
    
    total_weight = shelves["total_weight_gram"].where(shelves["book_rf_ids"] == rfids).dropna().item()
    
    if total_weight - sum(weights) >= 1000:
        shelf = shelves["shelf_id"].where(shelves["book_rf_ids"] == rfids).dropna().item()
        shelf_found(shelf)
        break
```

The code above took too long to run and didn't complete after a few minutes, so I interrupted it and tried the same methodology as I did in KQL:

First, I tested the ast.literal_eval() method to convert the "book_rf_ids" values to lists from strings:

In [9]:
first = shelves.book_rf_ids[0]
print(type(first))
first = ast.literal_eval(first)
print(type(first))

<class 'str'>
<class 'list'>


Once confirming that worked, I applied it to the entire DataFrame with the apply() method and used explode() to separate the "book_rf_ids" values into separate rows:

In [10]:
shelves.book_rf_ids = shelves.book_rf_ids.apply(ast.literal_eval)    
shelves_exploded = shelves.explode('book_rf_ids', ignore_index=True)

To prepare for merging the "books" DataFrame with the now-exploded "shelves" DataFrame, I made the "rf_id" column name match and tested that it worked:

In [11]:
shelves_exploded.rename(columns = {'book_rf_ids':'rf_id'}, inplace = True)
shelves_exploded

,shelf_id,rf_id,total_weight_gram
0,1395,e0e068b3ada347cb8ed99b5172fdedf4,54639
1,1395,d748c9ade1b74c1d9e1d31be9fa9491f,54639
2,1395,0e52ebe1b22d4906a6305543d86eaf34,54639
3,1395,9243bc7c7dd1405e81fad8c29054d3d7,54639
4,1395,b7668783b7ae4867b30f81c581a079e6,54639
...,...,...,...
325895,2814,2bd19381adbd4309a25749b41671bdaa,51528
325896,2814,8b11d7fd186149768717142ec8ba2c40,51528
325897,2814,e0306079fcd34ca9a42d631b38c0800d,51528
325898,2814,63754c3597224bd086bc2f826602fac7,51528


I then merged the two DataFrames on the RFIDs:

In [12]:
merged = pd.merge(shelves_exploded, books, on = "rf_id", how = "inner")

Finally, I aggregated the rows based on the Shelf ID. I kept the total weights as they were and summed the individual weights into one. Searching for the Shelf ID where the total weight was over 1000 grams more than the individually summed weights, I identified the correct shelf: 4242.

In [13]:
aggregation_functions = {'total_weight_gram': 'first', 'weight_gram': 'sum'}
weight_compare = merged.groupby('shelf_id', as_index=False).aggregate(aggregation_functions).reindex(columns=merged.columns)

shelf = weight_compare["shelf_id"].where((weight_compare["total_weight_gram"] - weight_compare["weight_gram"]) > 1000).dropna().item()
shelf_found(shelf)

Book found on shelf 4242!


### Conclusion: This problem takes less code to solve in KQL than Python.